# 🔄 Part 5: Iterative Workflows with LoopAgent
## Content Creation Studio Workshop

Welcome back! We've mastered sequential workflows (Part 4). Now: what if we need
**iterative refinement**? Content that improves itself until it meets quality standards?

Today we build a **quality improvement loop** - draft, critique, refine, repeat until perfect!

---

## 📚 The Content Creation Studio Playbook Series
- Part 1: Building Your First AI Agent ✅
- Part 2: Giving Your Agent Custom Tools ✅
- Part 3: Building Agent Teams with Agent-as-a-Tool ✅
- Part 4: Multi-Step Workflows with SequentialAgent ✅
- **Part 5: Building Iterative Workflows with LoopAgent** (You are here) 🎯
- Part 6: Efficient Workflows with ParallelAgent
- Part 7: The Capstone Project

---

## 🎓 New Concepts in This Part

In Part 5, we'll introduce these **new ADK concepts**:

1. **LoopAgent** - Workflow agent that repeats sub-agents until conditions are met
2. **ToolContext** - Special parameter for runtime control within tools
3. **tool_context.actions.escalate** - Flag to terminate loops from within tools
4. **Critique-Refine Pattern** - Architecture for autonomous quality improvement
5. **max_iterations Parameter** - Safety limit for loop cycles

Each concept will be clearly marked with 🆕 when first introduced!

---

## 💡 What is an Iterative Workflow?

### 🆕 NEW CONCEPT: LoopAgent (Workflow Agent)

> **What is LoopAgent?**  
> A LoopAgent is a workflow agent that repeatedly executes its sub-agents until either a condition is met or a maximum iteration count is reached. It's designed for processes that need iterative refinement - like improving content quality through successive critique and improvement cycles.
>
> **Key Characteristics**:
> - Repeats sub-agents multiple times
> - Exit conditions: `max_iterations` reached OR `escalate` flag set to `True`
> - Useful for quality improvement, optimization, and refinement tasks
> - Parameters: `sub_agents`, `max_iterations`
>
> **Exit Mechanisms**:
> 1. **Fixed iterations**: Loop runs exactly `max_iterations` times
> 2. **Conditional exit**: Tool sets `tool_context.actions.escalate = True`
>
> 📖 **Reference**: [Workflow Agents](https://google.github.io/adk-docs/agents/)

### Today's Loop:
```
Draft Content → Check Quality → (score < 70?) → Improve → Loop Back
                              → (score >= 70?) → Exit Loop → Done
```

### Key Pattern:
**Critique-Refine Cycle** - One agent evaluates, another improves, repeat until quality threshold is met!

## ⚙️ 1. Setup

In [ ]:
!pip install google-adk==1.19.0 -q

## 🔑 2. Authentication

In [ ]:
import os
from getpass import getpass

api_key = getpass('Enter your Google API Key: ')
os.environ['GOOGLE_API_KEY'] = api_key
print('✅ API Key configured!')

## 🛠️ 3. Define Custom Tools

### 🆕 NEW CONCEPT: ToolContext

> **What is ToolContext?**  
> ToolContext is a special parameter that can be added to custom tool functions to access runtime information and control workflow behavior. When a tool function includes `tool_context: ToolContext` as a parameter, ADK automatically passes in a context object that provides access to session state, runtime actions, and other execution details.
>
> **How to Use It**:
> ```python
> def my_tool(tool_context: ToolContext, other_param: str):
>     # Access session state
>     data = tool_context.state
>     # Control workflow execution
>     tool_context.actions.escalate = True
> ```
>
> **Capabilities**:
> - Access session state via `tool_context.state`
> - Control workflow behavior via `tool_context.actions`
> - Get runtime execution context
> - Signal special conditions (like loop termination)
>
> 📖 **Reference**: [Tool Context](https://google.github.io/adk-docs/tools/)

---

### 🆕 NEW CONCEPT: tool_context.actions.escalate

> **What is tool_context.actions.escalate?**  
> The `escalate` flag is a special boolean attribute that signals to a LoopAgent that it should terminate immediately, regardless of remaining iterations. When a tool sets `tool_context.actions.escalate = True`, the loop exits after the current agent completes.
>
> **Usage Pattern**:
> ```python
> def exit_loop(tool_context: ToolContext):
>     tool_context.actions.escalate = True
>     return {"result": "Loop terminated"}
> ```
>
> **When to Use**:
> - Quality thresholds met (content is good enough)
> - Goal achieved (problem solved)
> - Condition satisfied (target reached)
> - Early termination needed
>
> **How It Works**:
> 1. Agent calls tool that sets `escalate = True`
> 2. Current agent completes its execution
> 3. LoopAgent detects escalate flag
> 4. Loop terminates (even if iterations remain)
>
> 📖 **Reference**: [Tool Context](https://google.github.io/adk-docs/tools/)

---

We need two key tools:

1. **calculate_content_quality_score** - Evaluates content quality (returns score 0-100)
2. **exit_loop** - Terminates the iteration when quality threshold is met

### How exit_loop Works:
```python
def exit_loop(tool_context: ToolContext):
    tool_context.actions.escalate = True  # ← This stops the loop!
    return {"result": "Quality approved"}
```

The `escalate` flag signals to the LoopAgent: "We're done, exit now!"

In [ ]:
from google.adk.tools import ToolContext

def calculate_content_quality_score(
    word_count: int,
    readability_score: float,
    has_headings: bool,
    has_conclusion: bool
) -> dict:
    """
    Calculates overall content quality score (0-100).
    Threshold for approval: 70+
    """
    print(f"🔧 Calculating quality score...")
    
    # Word count scoring (optimal: 800-2000)
    if word_count < 500:
        word_score = 30
    elif word_count < 800:
        word_score = 60
    elif word_count <= 2000:
        word_score = 100
    else:
        word_score = 80
    
    # Readability scoring
    read_score = min(100, readability_score * 1.5) if readability_score > 0 else 40
    
    # Structure scoring
    structure_score = 0
    if has_headings:
        structure_score += 50
    if has_conclusion:
        structure_score += 50
    
    # Overall
    overall = (word_score * 0.3) + (read_score * 0.3) + (structure_score * 0.4)
    
    result = {
        "overall_score": round(overall, 2),
        "meets_threshold": overall >= 70
    }
    
    print(f"   Score: {result['overall_score']}/100 - {'✅ PASS' if result['meets_threshold'] else '❌ FAIL'}")
    return result

QUALITY_THRESHOLD_MET = "QUALITY_THRESHOLD_MET"

def exit_loop(tool_context: ToolContext):
    """Terminates loop when quality meets threshold."""
    print(f"🔧 Quality approved! Terminating loop...")
    tool_context.actions.escalate = True  # ← THE MAGIC HAPPENS HERE
    return {"result": "Quality threshold met"}

print("✅ Tools defined!")

## 🤖 4. Define the Agent Team

### 🆕 NEW CONCEPT: Critique-Refine Pattern

> **What is the Critique-Refine Pattern?**  
> The Critique-Refine Pattern is an architectural approach for building autonomous quality improvement systems. It consists of two complementary agents working in a loop: a **Checker Agent** that evaluates quality and identifies issues, and an **Improver Agent** that addresses those issues. The cycle repeats until quality standards are met.
>
> **Architecture Components**:
> 1. **Drafter Agent** (optional) - Creates initial version
> 2. **Checker Agent** - Evaluates quality, calculates scores, identifies problems
> 3. **Improver Agent** - Fixes issues, enhances quality, decides when to exit
>
> **The Cycle**:
> ```
> Draft → Check Quality → Identify Issues → Improve → Check Quality → ...
>                                                         ↓ (if quality met)
>                                                      Exit Loop
> ```
>
> **Benefits**:
> - Autonomous quality improvement
> - Clear separation of concerns (evaluation vs improvement)
> - Measurable quality gates
> - Self-correcting systems
>
> **Use Cases**:
> - Content refinement (our example)
> - Code optimization
> - Data cleaning
> - Iterative problem-solving
>
> 📖 **Reference**: [Workflow Agents](https://google.github.io/adk-docs/agents/)

---

Our iterative team has 3 members implementing the Critique-Refine Pattern:

1. **Drafter** - Creates initial content (runs once at the start)
2. **Quality Checker** - Critiques content and scores it (runs each loop iteration)
3. **Improver** - Refines content or exits if quality met (runs each loop iteration)

In [ ]:
from google.adk.agents import Agent
from google.adk.tools import google_search

# Agent 1: Drafter (runs once at start)
content_drafter_agent = Agent(
    name="content_drafter_agent",
    model="gemini-2.5-flash",
    instruction="""
    Write a blog post about: {{topic}}
    
    Create a draft (300-500 words) with:
    - Engaging intro
    - At least one H2 heading
    - A conclusion
    
    Output only the content in markdown.
    """,
    tools=[],
    output_key="current_content"
)

# Agent 2: Quality Checker (runs each loop iteration)
quality_checker_agent = Agent(
    name="quality_checker_agent",
    model="gemini-2.5-flash",
    instruction=f"""
    Analyze: {{{{current_content}}}}
    
    Your job:
    1. Count approximate words
    2. Estimate readability (60+ is good)
    3. Check for headings
    4. Check for conclusion
    
    Use `calculate_content_quality_score` tool.
    
    Then:
    - IF overall_score >= 70: respond '{QUALITY_THRESHOLD_MET}'
    - ELSE: respond 'Score: [X]. Issues: [specific problems]'
    """,
    tools=[calculate_content_quality_score],
    output_key="quality_feedback"
)

# Agent 3: Improver (runs each loop iteration)
content_improver_agent = Agent(
    name="content_improver_agent",
    model="gemini-2.5-flash",
    instruction=f"""
    Current content: {{{{current_content}}}}
    Feedback: {{{{quality_feedback}}}}
    
    - IF feedback is '{QUALITY_THRESHOLD_MET}': call `exit_loop` immediately
    - ELSE: improve based on issues:
      * Expand if short
      * Simplify if complex
      * Add headings if missing
      * Add conclusion if missing
    
    Output the COMPLETE improved content.
    """,
    tools=[exit_loop],
    output_key="current_content"  # ← Overwrites for next iteration!
)

print("🧞 Agent team created!")

## 🔄 5. Build the Loop

### max_iterations Parameter

> **What is max_iterations?**  
> The `max_iterations` parameter on LoopAgent sets a **safety limit** for the maximum number of times the loop can execute. This prevents infinite loops and ensures your workflow terminates even if the exit condition is never met.
>
> **Usage**:
> ```python
> LoopAgent(
>     sub_agents=[checker, improver],
>     max_iterations=3  # Loop runs at most 3 times
> )
> ```
>
> **Exit Conditions**:
> - Loop exits when `max_iterations` is reached, OR
> - Loop exits early if `tool_context.actions.escalate = True`
>
> **Best Practices**:
> - Set a reasonable limit (3-5 for most cases)
> - Too low: May not reach quality threshold
> - Too high: Wastes resources, slow execution
> - Always include escalate logic for early exit

---

Create the **LoopAgent** with our critique-refine cycle:

```python
LoopAgent(
    sub_agents=[quality_checker, content_improver],
    max_iterations=3
)
```

The loop runs **up to 3 times**, but exits early if the `exit_loop` tool is called (when quality threshold is met).

In [ ]:
from google.adk.agents import SequentialAgent, LoopAgent

# The iterative quality loop
quality_improvement_loop = LoopAgent(
    name="quality_improvement_loop",
    sub_agents=[quality_checker_agent, content_improver_agent],
    max_iterations=3
)

# Final presenter
final_presenter = Agent(
    name="final_presenter",
    model="gemini-2.5-flash",
    instruction="""
    Present the final content: {{current_content}}
    
    Add a brief intro: "✅ **Final Approved Content** (Quality threshold met)"
    Then show the complete content.
    """,
    tools=[]
)

# Complete workflow: Draft → Loop → Present
quality_workflow = SequentialAgent(
    name="quality_workflow",
    sub_agents=[
        content_drafter_agent,
        quality_improvement_loop,
        final_presenter
    ]
)

print("✅ Iterative workflow created!")

## 🚀 6. Build Execution Engine

In [ ]:
from IPython.display import display, Markdown
from google.adk.sessions import InMemorySessionService, Session
from google.adk.runners import Runner
from google.genai.types import Content, Part

session_service = InMemorySessionService()
user_id = "adk_content_creator_001"

async def run_agent_query(agent, query, session, user_id, state_delta=None):
    print(f"\n🚀 Running: '{agent.name}'...")
    
    runner = Runner(agent=agent, session_service=session_service, app_name=agent.name)
    
    final_response = ""
    try:
        async for event in runner.run_async(
            user_id=user_id,
            session_id=session.id,
            new_message=Content(parts=[Part(text=query)], role="user"),
            state_delta=state_delta
        ):
            if event.is_final_response():
                final_response = event.content.parts[0].text
    except Exception as e:
        final_response = f"Error: {e}"
    
    print("\n" + "-"*50)
    display(Markdown(final_response))
    print("-"*50)
    
    return final_response

print("✅ Engine ready!")

## ✨ 7. Run the Iterative Workflow!

Watch the quality improvement cycle:
1. 📝 Draft initial content
2. 🔍 Check quality (iteration 1)
3. ✏️ Improve (iteration 1)
4. 🔍 Check quality (iteration 2)
5. ✅ Exit loop (quality met!)
6. 📄 Present final

In [ ]:
async def run_quality_workflow():
    session = await session_service.create_session(
        app_name=quality_workflow.name,
        user_id=user_id
    )
    
    topic = "The benefits of meditation for busy professionals"
    session.state["topic"] = topic
    
    query = f"Create high-quality content about: {topic}"
    print(f"👤 User: {query}\n")
    
    await run_agent_query(
        quality_workflow, query, session, user_id,
        state_delta={"topic": topic}
    )

await run_quality_workflow()

## 🎉 Recap: What We've Learned

This was a powerful step in building autonomous quality improvement systems!

### Core Concepts Introduced:

1. **🆕 LoopAgent (Workflow Agent)** - Repeats sub-agents until condition met  
   📖 [Workflow Agents](https://google.github.io/adk-docs/agents/)

2. **🆕 ToolContext** - Special parameter for runtime control in tools  
   Usage: `def my_tool(tool_context: ToolContext):`  
   📖 [Tool Context](https://google.github.io/adk-docs/tools/)

3. **🆕 tool_context.actions.escalate** - Terminates loops from within tools  
   Usage: `tool_context.actions.escalate = True`  
   📖 [Tool Context](https://google.github.io/adk-docs/tools/)

4. **🆕 Critique-Refine Pattern** - Autonomous quality improvement architecture  
   Pattern: Checker Agent → Improver Agent → Loop until quality met

5. **🆕 max_iterations Parameter** - Safety limit for loop cycles  
   Usage: `LoopAgent(sub_agents=[...], max_iterations=3)`

### Key Takeaways:

- **Iterative Refinement**: Content improves itself through cycles
- **Quality Gates**: Threshold-based autonomous approval
- **State Overwriting**: `output_key` gets updated each iteration
- **Dual Exit Strategy**: max_iterations OR escalate flag
- **Self-Improving Systems**: Agents critique and refine their own work

### Architecture Pattern:
```
Draft (output_key="current_content")
  ↓
Loop (max_iterations=3):
  ├─ Check Quality (reads {{current_content}}, output_key="quality_feedback")
  │    ↓
  └─ Improve (reads {{quality_feedback}})
      ├─ If quality met → tool_context.actions.escalate = True → Exit Loop
      └─ If not met → output_key="current_content" (overwrites) → Loop again
           ↓
Present Final Content
```

### The Power of LoopAgent:

The `current_content` variable gets **overwritten** each iteration, allowing the content to evolve organically until it meets quality standards - all autonomously!

---

## 🚀 What's Next?

We can now create self-improving workflows! But what if we need **parallel execution**? What if we want to create a blog post, social content, email newsletter, and SEO metadata **all at once**?

In **Part 6**, we'll learn about **ParallelAgent** - a workflow agent that runs multiple sub-agents concurrently for maximum efficiency!

### Preview of Part 6 Concepts:
- 🆕 ParallelAgent (Workflow Agent)
- 🆕 Fan-Out/Fan-In Pattern
- 🆕 Intake Agent Pattern
- 🆕 Session State Management

See you in Part 6! 🚀